## Setup

Run this cell first to provision a cloud machine and set up Drake. 

This may take a minute or two the first time, but once a machine is set up it should be good for about 12 hours. 

In [1]:
import importlib
import sys
import os
from urllib.request import urlretrieve
import subprocess
import shutil

drake_url = "https://drake-packages.csail.mit.edu/tmp/drake-0.27.0-pip-bionic.tar.gz"
assert 'google.colab' in sys.modules, "This notebook is meant to be run in google colab!"

if importlib.util.find_spec('pydrake') is None:
    # We're in colab and don't have pydrake, so install it on the cloud machine.
    if os.path.isdir('/opt/drake'):
        shutil.rmtree('/opt/drake')
    print("Installing Drake")
    urlretrieve(drake_url, 'drake.tar.gz')
    subprocess.run(['mkdir', '/opt/drake'])
    subprocess.run(['tar', '-xzf', 'drake.tar.gz', '-C', '/opt/drake'], check=True)
    
    print("Installing other dependencies")
    subprocess.run(["pip3", "install", "meshcat"])
    subprocess.run(['apt-get', 'update', '-o', 'APT::Acquire::Retries=4', '-qq'], check=True)
    with open("/opt/drake/share/drake/setup/packages-bionic.txt", "r") as f:
        packages = f.read().splitlines()
    subprocess.run(['apt-get', 'install', '-o',
                    'APT::Acquire::Retries=4', '-o', 'Dpkg::Use-Pty=0',
                    '-qy', '--no-install-recommends'] + packages,
                    check=True)
    
    v = sys.version_info
    path = f"/opt/drake/lib/python{v.major}.{v.minor}/site-packages"
    if importlib.util.find_spec('pydrake') is None:
        sys.path.append(path)

# Clone our github repo
target_path = '/opt/passivity_cbf_demo'
if not os.path.isdir(target_path):
    print("Cloning github repo")
    subprocess.run(['git','clone','https://github.com/vincekurtz/passivity_cbf_demo.git',target_path])
sys.path.append(target_path)

print("Done!")


Installing Drake
Installing other dependencies
Cloning github repo
Done!


## About This Notebook

While you're waiting, here's what this is all about...

## Simulation Setup



Choose what controller we'll use and what constraints are active

TODO: add summary of each method.

In [15]:
import ipywidgets as widgets

ctrl_radio = widgets.RadioButtons(
    options=["Unconstrained","Standard","Passivity Guaranteed"],
    value="Passivity Guaranteed"
)
display(ctrl_radio)



RadioButtons(index=2, options=('Unconstrained', 'Standard', 'Passivity Guaranteed'), value='Passivity Guarante…

Choose what type of constraints to apply, if any.

TODO: add summary of each method. 

In [16]:
cons_radio = widgets.RadioButtons(
    options=["Singularity Avoidance", "Joint Limits", "None"],
    value="Singularity Avoidance"
)
display(cons_radio)

RadioButtons(options=('Singularity Avoidance', 'Joint Limits', 'None'), value='Singularity Avoidance')

## Run the Simulation!